#6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%reset-f
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy

In [ ]:
def hot_enc(insts):
  y_en = np.zeros([len(insts[:,0]),3])
  y_en[:,0][insts[:,-1]==1] = 1
  y_en[:,1][insts[:,-1]==2] = 1
  y_en[:,2][insts[:,-1]==3] = 1
  return y_en
def perfor(mat,m):
  per = []
  for i in range(3):
    per.append( mat[i,i]/(mat[i,0]+mat[i,1]+mat[i,2]))
  per.append((mat[0,0]+mat[1,1]+mat[2,2])/m)
  return per

def input_to_hidden(x,Win,t):
  if t==1:
    a = np.dot(x, Win)
    b1=1-np.exp(-a) #hyperbolic tan
    b2=1+np.exp(-a)
    a=b1/b2
  if t==2:
    f = np.dot(x, Win) #gaussian 
    a= 1 / (1 + np.exp(-f))
  return a
def predict(x,Wout,t):
    x = input_to_hidden(x,Wout,t)
    y = np.dot(x, Wout)
    return y

In [ ]:
path = '/content/drive/MyDrive/ML/data5.xlsx'
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #number of instances
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n= len(insts[0,:])-1 # no of features
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std() #Normalize the data
np.random.shuffle(insts)

K = 5 # no of folds
overall_accu_max = 0
accu_class1_max = 0
accu_class2_max = 0
accu_class3_max = 0
for lamb in range(10,100,5):
  for i in range(10,90,5):
    overall_accu = 0
    accu_class1 = 0
    accu_class2 = 0
    accu_class3 = 0
    for k in range(K):
      insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
      insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
      m_tr = len(insts_tr[:,0])
      m_te = len(insts_te[:,0]) 
      n_h  = i #no of hidden neurons
      y_tr = hot_enc(insts_tr)
      v = np.random.rand(n,n_h)
      h = input_to_hidden(insts_tr[:,0:n],v,1)
      w = np.dot(np.linalg.pinv(np.dot(h.T,h)+ lamb) , np.dot(h.T,y_tr) )
      #testing
      h_te = input_to_hidden(insts_te[:,0:n],v,1)
      
      y_pre = np.dot(h_te,w)
      y_p = np.zeros(m_te)
      for i in range(m_te):
        y_p[i] = np.argmax(y_pre[i])+1
      confmat = pd.crosstab(insts_te[:,-1], y_p)
      confmat = np.asarray(confmat)
      per = perfor(confmat,m_te)
      overall_accu += per[3]
      accu_class1 += per[0]
      accu_class2 += per[1]
      accu_class3 += per[2]
    if(overall_accu>overall_accu_max):
      overall_accu_max = overall_accu
    if(accu_class1>accu_class1_max):
      accu_class1_max = accu_class1
    if(accu_class2>accu_class2_max):
      accu_class2_max = accu_class2
    if(accu_class3>accu_class3_max):
      accu_class3_max = accu_class3
print('accu_class1:',accu_class1_max/K,'\naccu_class2:',accu_class2_max/K,'\naccu_class3:',accu_class3_max/K,'\noverall_accu:',overall_accu_max/K) 

accuracy_class1: 0.9233333333333332 
accuracy_class2: 1.0 
accuracy_class3: 0.9875 
overall_accuracy: 0.9522648083623693


In [ ]:
#gaussian
#tan hyperbolic
K = 5 # no of folds
overall_accu_max = 0
accu_class1_max = 0
accu_class2_max = 0
accu_class3_max = 0
for lamb in range(10,100,5):
  for i in range(10,90,5):
    overall_accu = 0
    accu_class1 = 0
    accu_class2 = 0
    accu_class3 = 0
    for k in range(K):
      insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
      insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
      m_tr = len(insts_tr[:,0])
      m_te = len(insts_te[:,0]) 
      n_h  = i #no of hidden neurons
      y_tr = hot_enc(insts_tr)
      v = np.random.rand(n,n_h)
      h = input_to_hidden(insts_tr[:,0:n],v,2)
      w = np.dot(np.linalg.pinv(np.dot(h.T,h)+ lamb) , np.dot(h.T,y_tr) )
      #testing
      h_te = input_to_hidden(insts_te[:,0:n],v,2)
      
      y_pre = np.dot(h_te,w)
      y_p = np.zeros(m_te)
      for i in range(m_te):
        y_p[i] = np.argmax(y_pre[i])+1
      confmat = pd.crosstab(insts_te[:,-1], y_p)
      confmat = np.asarray(confmat)
      per = perfor(confmat,m_te)
      overall_accu += per[3]
      accu_class1 += per[0]
      accu_class2 += per[1]
      accu_class3 += per[2]
    if(overall_accu>overall_accu_max):
      overall_accu_max = overall_accu
    if(accu_class1>accu_class1_max):
      accu_class1_max = accu_class1
    if(accu_class2>accu_class2_max):
      accu_class2_max = accu_class2
    if(accu_class3>accu_class3_max):
      accu_class3_max = accu_class3
print('accu_class1:',accu_class1_max/K,'\naccu_class2:',accu_class2_max/K,'\naccu_class3:',accu_class3_max/K,'\noverall_accu:',overall_accu_max/K) 

accuracy_class1: 0.9266666666666665 
accuracy_class2: 1.0 
accuracy_class3: 0.9875 
overall_accuracy: 0.956910569105691
